In [1]:
import numpy as np
import pandas as pd
import os

In [3]:
!git clone https://github.com/ultralytics/yolov5

Cloning into 'yolov5'...
remote: Enumerating objects: 19, done.
remote: Counting objects: 100% (19/19), done.
remote: Compressing objects: 100% (16/16), done.
remote: Total 854 (delta 5), reused 10 (delta 3), pack-reused 835
Receiving objects: 100% (854/854), 3.39 MiB | 24.10 MiB/s, done.
Resolving deltas: 100% (556/556), done.


In [4]:
!ls

sample_data  yolov5


In [5]:
!mv yolov5/* ./

In [6]:
!pip install -r requirements.txt

  Cloning https://github.com/cocodataset/cocoapi.git to /tmp/pip-req-build-xvih7om_
  Running command git clone -q https://github.com/cocodataset/cocoapi.git /tmp/pip-req-build-xvih7om_
     |████████████████████████████████| 20.4MB 1.3MB/s 
     |████████████████████████████████| 276kB 44.8MB/s 
  Created wheel for PyYAML: filename=PyYAML-5.3.1-cp36-cp36m-linux_x86_64.whl size=44621 sha256=b60e83130ab2116c06de34a6e9a93a824757b7c5f3351bf25f16d17bef207e14
  Stored in directory: /root/.cache/pip/wheels/a7/c1/ea/cf5bd31012e735dc1dfea3131a2d5eae7978b251083d6247bd
  Created wheel for pycocotools: filename=pycocotools-2.0-cp36-cp36m-linux_x86_64.whl size=266990 sha256=cee6903706df9447ee68f618002c4f11c6785f37c24d57a5c611a92bb9e06197
  Stored in directory: /tmp/pip-ephem-wheel-cache-8kha2ego/wheels/90/51/41/646daf401c3bc408ff10de34ec76587a9b3ebfac8d21ca5c3a
Successfully built PyYAML pycocotools
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incom

In [33]:
MAIN_DIR = '/content/drive/My Drive/Git/GlobalWheatDetection'

In [35]:
df = pd.read_csv(f'{MAIN_DIR}/input/train.csv')

# 学習データのbboxコラム[x, y, w, h]（文字列）を、配列にする
# np.stackで、２次元配列化1行1正解データ
bboxs = np.stack(df['bbox'].apply(lambda x: np.fromstring(x[1:-1], sep=',')))
for i, column in enumerate(['x', 'y', 'w', 'h']):
    # dfに追加
    df[column] = bboxs[:, i]
df.drop(columns=['bbox'], inplace=True)
df['x_center'] = df['x'] + df['w']/2
df['y_center'] = df['y'] + df['h']/2
df['classes'] = 0
from tqdm.auto import tqdm
import shutil as sh
df = df[['image_id', 'x', 'y', 'w', 'h', 'x_center', 'y_center', 'classes']]
df.head()

,image_id,x,y,w,h,x_center,y_center,classes
0,b6ab77fd7,834.0,222.0,56.0,36.0,862.0,240.0,0
1,b6ab77fd7,226.0,548.0,130.0,58.0,291.0,577.0,0
2,b6ab77fd7,377.0,504.0,74.0,160.0,414.0,584.0,0
3,b6ab77fd7,834.0,95.0,109.0,107.0,888.5,148.5,0
4,b6ab77fd7,26.0,144.0,124.0,117.0,88.0,202.5,0


In [36]:
index = list(set(df.image_id))
print(index)

['4883da2a8', '0e0809a3d', 'fe029eca4', '3d5f3124e', '178d835fd', 'ebc1c07c1', 'a5dde818e', '7e18f16b8', '6c66bb62d', '3d765f241', '3ce598c96', '78edb3987', '7fcfe0ae1', 'dfbc049cd', '086b6a39d', '773c33268', '0dc2de69d', '7b0840687', '0fd4ca0d4', '63c658201', '6d1ff0f34', 'd1ea44cc7', '6902bbaf2', '0aef2f67e', 'f985e704d', '3d39ed7cf', 'dc3b08dc0', 'a57808743', 'fbecfeba8', '48c72c991', 'b18b34ed2', 'aa644920c', 'b28fb55cc', '4ecd10d42', 'df1ba5253', 'e69cbaece', '999d9d27d', 'eb256e6be', 'ca5e51e59', '39cb95bc9', 'e14f4de23', '18f8b24f1', '3d9302a17', '0ce3a22a3', 'ce3999eb9', '224017b57', '69fc3d3ff', 'b0daf69ce', 'feac3a701', '278038027', 'e935b9eea', 'ae876f3ef', '7347e16e8', 'b6e406cd1', 'ca3101e66', 'd688932d4', 'c6a299f4d', '7e75b3e43', 'e290bc809', 'd17efcdba', '7fd1d7a57', 'e38514029', '09ba65605', '8405c37aa', 'e837f4e67', '1e58125ec', '1d2950c50', '7c7d269c8', 'e2d034e7d', '87196861b', '1aa099fd7', 'e2ac8324c', 'ef156f858', 'c64a4006c', 'e224a64ba', '3fd04f5cd', '5a5631a48'

In [39]:
source = 'train'
if True:
    for fold in [0]:
        # 全体データの1/foldから2/foldまでをバッチで取得
        val_index = index[len(index) * fold // 5:len(index) * (fold+1)//5]
        for name, mini in tqdm(df.groupby('image_id')):
            if name in val_index:
                save_path = f'{MAIN_DIR}/output/fold{fold}/labels/val2017/'
            else:
                save_path = f'{MAIN_DIR}/output/fold{fold}/labels/train2017/'
            
            if not os.path.exists(save_path):
                os.makedirs(save_path)                


SyntaxError: ignored